In [1]:
!pip install diffusers transformers accelerate scipy safetensors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.7 MB/s eta 0:00:00


In [2]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch

model_id = "stabilityai/stable-diffusion-2-base"

# Use the Euler scheduler here instead
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]  


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [3]:
import numpy as np
"""
Linear interpolate between start frame and end frame

start: shape (tokens, feat_dim), text embedding of the starting frame
end: shape (tokens, feat dim), text embedding of the ending frame
num_frames: number of frames to be produced between the start and end frames

return: shape (num_frames, tokens, feat dim), embeddings of interpolated text
"""
def linear_interpolate(start, end, num_frames):
  if len(start.shape) == 2:
    start = torch.unsqueeze(start, dim=0)
  if len(end.shape) == 2:
    end = torch.unsqueeze(end, dim=0)

  steps = np.linspace(0., 1., num_frames+2)[1:-1]
  prompt_embeddings_interp = torch.zeros_like(start)
  for step in steps:
    prompt_embeddings_interp = torch.cat((prompt_embeddings_interp,(1 - step)*start + step*end), dim=0)
  prompt_embeddings_interp = prompt_embeddings_interp[1:,:,:]

  return prompt_embeddings_interp

if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

num_images_per_prompt = 1
do_classifier_free_guidance = True

prompt_embeddings_start = pipe._encode_prompt("start", device, num_images_per_prompt, do_classifier_free_guidance)[1,:,:]
prompt_embeddings_end = pipe._encode_prompt("end", device, num_images_per_prompt, do_classifier_free_guidance)[1,:,:]

print(linear_interpolate(prompt_embeddings_start, prompt_embeddings_end, 5).shape)

torch.Size([5, 77, 1024])
